In [8]:
Dataset = "FoCus"                                  # Synthetic-PersonaChat, Blended Skill Talk, PEC, ConvAI2, FoCus, IT-ConvAI2
LLM_name = "Llama3-1-8B-Instruct"                   # Mistral-7B-Instruct, Llama3-1-8B-Instruct, Qwen2-7B-Instruct, Gemma-7B-Instruct

# _COT = ""
_COT = "-COT"

In [9]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

# ds = pd.read_csv(f'New Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds

Shape: (1000, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,"{\n ""reasoning"": ""I personalized the response...",8.849191
1,"{\n ""reasoning"": ""I personalized the response...",8.896329
2,"{\n ""reasoning"": ""I personalized the response...",8.867732
3,"{\n ""reasoning"": ""I personalized the response...",9.021704
4,"{\n ""reasoning"": ""I personalized the response...",8.908265
...,...,...
995,"{\n ""reasoning"": ""I personalized the response...",8.841254
996,"{\n ""reasoning"": ""I personalized the response...",8.828058
997,"{\n ""reasoning"": ""I personalized the response...",8.916159
998,"{\n ""reasoning"": ""I considered the conversati...",8.955311


In [10]:
print(ds.iloc[167]["gen_response"])

{
  "reasoning": "I personalized the response by considering the user's interest in pearls and cemeteries, as well as their desire to visit Australia and see a heritage site. I also took into account the conversation context, which mentioned Thursday Island Cemetery and its historical significance. I reflected on the user's Japanese background and how it relates to the cemetery's history, and tailored my response to provide more information about the cemetery's importance.",
  "response": "Thursday Island Cemetery is a significant heritage site in Queensland, Australia, that showcases the history of Japanese divers who came to the region to find pearls. The cemetery is home to over 600 graves with Japanese names, highlighting the importance of this site in understanding the cultural exchange between Japan and Australia. A visit to this cemetery would not only be a poignant experience but also a unique opportunity to learn about the region's history and the lives of the people who came 

In [11]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    
    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     50
response_time     0
dtype: int64
(1000, 2)


,gen_response,response_time
0,"Nazareth House is a truly remarkable place, an...",8.849191
1,I'm glad you're interested in historic houses!...,8.896329
2,The Marion Palace Theatre's architecture is a ...,8.867732
3,Technische Universität Darmstadt is a great ch...,9.021704
4,None,8.908265
...,...,...
995,I think you're referring to the museum come ph...,8.841254
996,It's great to know that Mahasthangarh has such...,8.828058
997,Armagh County Museum is a must-visit destinati...,8.916159
998,The Nyanga National Park is a great destinatio...,8.955311


In [12]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)